# **Chapter 3: Assigning Roles (Role Prompting)**

---
**Lesson:**

Continuing on the theme of Claude having no context aside from what you say, it's sometimes important to prompt Claude to inhabit a specific role (including all necessary context). This is also known as role prompting. The more detail to the role context, the better.

Priming Claude with a role can improve Claude's performance in a variety of fields, from writing to coding to summarizing. It's like how humans can sometimes be helped when told to ""think like a ______""." 

---

First we will setup our dependencies

In [ ]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

#Set up model inference modifiers, and ensure Claude v2 is being called from Bedrock 
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

# **Examples:**

In the example below (3.1), we see that without priming, Claude can provide a decent if somewhat disjointed answer when asked to explain the impact of greenhouse gases. 

However, when we prime Claude to be a highly experienced climate scientist, Claude's response becomes more organized and detailed (3.2). 

> *Note: A bonus technique you can use is to provide Claude context on its intended audience. Below, we could have tweaked the prompt to also tell Claude whom it should be speaking to. "Explain the impact of greenhouse gasses" produces very different results in terms of tone from "Explain the impact of greenhouse gasses to a third grade audience."*

**Example 3.1 - Greenhouse**

In [ ]:
#Prompt without role assignment
response = textgen_llm("""

Human: Please explain the impact of greenhouse gases on the planet.

Assistant:""")

print_ww(response)

**Example 3.2 - Organized greenhouse**		

In [ ]:
#Prompt with role assignment
response = textgen_llm("""

You are a highly experienced climate scientist. You respond in a succinct and scientific manner with multiple data points.

Human:  Please explain the impact of greenhouse gases.

Assistant:""")

print_ww(response)

**Example 3.3 - Logical?**

Here's another example. Let's ask Claude to solve this logical problem

In [ ]:
#Prompt without role assignment
response = textgen_llm("""

Human: Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don't know if Anne is married. Is a married person looking at an unmarried person?

Assistant:""")

print_ww(response)

**Example 3.4 - Logical!**

The answer above is surface level reasoning based on the information given. Let us give Claude a role as a "Logic Bot" and see how it's response changes

In [ ]:
#Prompt with role assignment
response = textgen_llm("""

You are a logic bot designed to answer complex logic problems. You think of all possibilities when solving problems

Human: 
Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don't know if Anne is married. Is a married person looking at an unmarried person?

Assistant:""")

print_ww(response)

# **Exercise**

The following exercise will need you to manipulate the prompt to get the desired output

**Exercise 3.1 - Math Correction**

In some instances, Claude may struggle with mathematics, even simple mathematics. Below, Claude incorrectly assesses the math problem as correctly solved, even though there's an obvious arithmetic mistake in the second step.

Adapt the prompt below to make Claude grade the solution as "incorrectly" solved, rather than "correctly" solved. 

> *"Note: XML tags (<> to open, </> to close) are a common and highly effective way to organize information within a prompt. Sometimes it also helps to ask Claude to think about its answer first before responding. 
We will cover these two topics in greater depth in Chapters 4 and 6 respectively."*

In [ ]:
response = textgen_llm("""

Human: Is the equation below solved correctly? 

2x - 3 = 9
2x = 6 
x = 3  

Assistant:""")

print_ww(response)

# Chapter 3 - END.